In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import gillespy2
import dask
import copy
import json
import time
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
#from tsfresh.feature_extraction.settings import MinimalFCParameters
from dask.distributed import Client
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sciope.utilities.priors import uniform_prior
from torch.utils.data import DataLoader, TensorDataset

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Model for data generation

In [60]:
# Use the model definition below
class Vilar_Oscillator(gillespy2.Model):
    def __init__(self, parameter_values=None):
        gillespy2.Model.__init__(self, name="Vilar_Oscillator")
        self.volume = 1

        # Parameters
        self.add_parameter(gillespy2.Parameter(name="alpha_a", expression=50))
        self.add_parameter(gillespy2.Parameter(name="alpha_a_prime", expression=500))
        self.add_parameter(gillespy2.Parameter(name="alpha_r", expression=0.01))
        self.add_parameter(gillespy2.Parameter(name="alpha_r_prime", expression=50))
        self.add_parameter(gillespy2.Parameter(name="beta_a", expression=50))
        self.add_parameter(gillespy2.Parameter(name="beta_r", expression=5))
        self.add_parameter(gillespy2.Parameter(name="delta_ma", expression=10))
        self.add_parameter(gillespy2.Parameter(name="delta_mr", expression=0.5))
        self.add_parameter(gillespy2.Parameter(name="delta_a", expression=1))
        self.add_parameter(gillespy2.Parameter(name="delta_r", expression=0.2))
        self.add_parameter(gillespy2.Parameter(name="gamma_a", expression=1))
        self.add_parameter(gillespy2.Parameter(name="gamma_r", expression=1))
        self.add_parameter(gillespy2.Parameter(name="gamma_c", expression=2))
        self.add_parameter(gillespy2.Parameter(name="theta_a", expression=50))
        self.add_parameter(gillespy2.Parameter(name="theta_r", expression=100))

        # Species
        self.add_species(gillespy2.Species(name="Da", initial_value=1, mode="discrete"))
        self.add_species(gillespy2.Species(name="Da_prime", initial_value=0, mode="discrete"))
        self.add_species(gillespy2.Species(name="Ma", initial_value=0, mode="discrete"))
        self.add_species(gillespy2.Species(name="Dr", initial_value=1, mode="discrete"))
        self.add_species(gillespy2.Species(name="Dr_prime", initial_value=0, mode="discrete"))
        self.add_species(gillespy2.Species(name="Mr", initial_value=0, mode="discrete"))
        self.add_species(gillespy2.Species(name="C", initial_value=10, mode="discrete"))
        self.add_species(gillespy2.Species(name="A", initial_value=10, mode="discrete"))
        self.add_species(gillespy2.Species(name="R", initial_value=10, mode="discrete"))

        # Reactions
        self.add_reaction(gillespy2.Reaction(name="r1", reactants={'Da_prime': 1}, products={'Da': 1}, rate=self.listOfParameters["theta_a"]))
        self.add_reaction(gillespy2.Reaction(name="r2", reactants={'Da': 1, 'A': 1}, products={'Da_prime': 1}, rate=self.listOfParameters["gamma_a"]))
        self.add_reaction(gillespy2.Reaction(name="r3", reactants={'Dr_prime': 1}, products={'Dr': 1}, rate=self.listOfParameters["theta_r"]))
        self.add_reaction(gillespy2.Reaction(name="r4", reactants={'Dr': 1, 'A': 1}, products={'Dr_prime': 1}, rate=self.listOfParameters["gamma_r"]))
        self.add_reaction(gillespy2.Reaction(name="r5", reactants={'Da_prime': 1}, products={'Da_prime': 1, 'Ma': 1}, rate=self.listOfParameters["alpha_a_prime"]))
        self.add_reaction(gillespy2.Reaction(name="r6", reactants={'Da': 1}, products={'Da': 1, 'Ma': 1}, rate=self.listOfParameters["alpha_a"]))
        self.add_reaction(gillespy2.Reaction(name="r7", reactants={'Ma': 1}, products={}, rate=self.listOfParameters["delta_ma"]))
        self.add_reaction(gillespy2.Reaction(name="r8", reactants={'Ma': 1}, products={'A': 1, 'Ma': 1}, rate=self.listOfParameters["beta_a"]))
        self.add_reaction(gillespy2.Reaction(name="r9", reactants={'Da_prime': 1}, products={'Da_prime': 1, 'A': 1}, rate=self.listOfParameters["theta_a"]))
        self.add_reaction(gillespy2.Reaction(name="r10", reactants={'Dr_prime': 1}, products={'Dr_prime': 1, 'A': 1}, rate=self.listOfParameters["theta_a"]))
        self.add_reaction(gillespy2.Reaction(name="r11", reactants={'A': 1}, products={}, rate=self.listOfParameters["gamma_c"]))
        self.add_reaction(gillespy2.Reaction(name="r12", reactants={'A': 1, 'R': 1}, products={'C': 1}, rate=self.listOfParameters["gamma_c"]))
        self.add_reaction(gillespy2.Reaction(name="r13", reactants={'Dr_prime': 1}, products={'Dr_prime': 1, 'Mr': 1}, rate=self.listOfParameters["alpha_r_prime"]))
        self.add_reaction(gillespy2.Reaction(name="r14", reactants={'Dr': 1}, products={'Dr': 1, 'Mr': 1}, rate=self.listOfParameters["alpha_r"]))
        self.add_reaction(gillespy2.Reaction(name="r15", reactants={'Mr': 1}, products={}, rate=self.listOfParameters["delta_mr"]))
        self.add_reaction(gillespy2.Reaction(name="r16", reactants={'Mr': 1}, products={'Mr': 1, 'R': 1}, rate=self.listOfParameters["beta_r"]))
        self.add_reaction(gillespy2.Reaction(name="r17", reactants={'R': 1}, products={}, rate=self.listOfParameters["delta_r"]))
        self.add_reaction(gillespy2.Reaction(name="r18", reactants={'C': 1}, products={'R': 1}, rate=self.listOfParameters["delta_a"]))

        # Timespan
        self.timespan(np.linspace(0, 200, 201))

In [61]:
vilar_model = Vilar_Oscillator()

In [ ]:
def configure_simulation():
    solver = gillespy2.SSACSolver(model=vilar_model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

In [ ]:
kwargs = configure_simulation()
fixed_data = vilar_model.run(**kwargs)
fixed_data.plotplotly()

## Prior Distribution

In [62]:
default_param = np.array(list(vilar_model.listOfParameters.items()))[:, 1]

bound = []
for exp in default_param:
    bound.append(float(exp.expression))

# Set the bounds
bound = np.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

In [63]:
N = 10000
samples_delayed = uni_prior.draw(N)
samples ,= dask.compute(samples_delayed)
samples = np.asarray(samples)
samples = samples.reshape(N, len(bound))

## Simulator

In [64]:
parameter_names = ['alpha_a', 'alpha_a_prime', 'alpha_r', 'alpha_r_prime', 'beta_a', 'beta_r', 'delta_ma',
                    'delta_mr', 'delta_a', 'delta_r', 'gamma_a', 'gamma_r', 'gamma_c', 'theta_a', 'theta_r']

solver = gillespy2.solvers.SSACSolver(model=vilar_model)

def sim_vilar(params, model=vilar_model):
    res = model.run(
            number_of_trajectories = 1,
            solver = solver,
            variables = {parameter_names[i] : np.asarray(params[i]) for i in range(len(parameter_names))})

    tot_res = np.zeros((len(res), 3, len(res[0]['C'])))
    for i in range(len(res)):
        tot_res[i, 0, :] = res[i]['C']
        tot_res[i, 1, :] = res[i]['A']
        tot_res[i, 2, :] = res[i]['R']

    return tot_res

In [65]:
params = np.asarray([50.0, 500.0, 0.01, 50.0, 50.0, 5.0, 10.0, 0.5, 1.0, 0.2, 1.0, 1.0, 2.0, 50.0, 100.0])

In [66]:
len(sim_vilar(params=params))

1

## Generating training samples based on prior distribution

In [67]:
responses, s = [], []
num_species = 3
num_timestamps = 201

for i in range(N):
    current_sample = np.squeeze(samples[i,:].reshape(1, len(samples[i,:])))
    lazy_response = dask.delayed(sim_vilar)(current_sample)
    responses.append(lazy_response)
    s.append(current_sample)
    #print(i)

# dask compute
computed_responses ,= dask.compute(responses)

# get it in the right shape sciope shape - N x S x T
#ts = np.asarray(computed_responses)
#ts = ts.reshape(N, num_species, num_timestamps)

In [68]:
'''
dic = {}
dic['c'], dic['a'], dic['r'] = [], [], []
for i in range(len(computed_responses)):
    for j in range(len(computed_responses[i][0])):
        dic['c'].append(list(computed_responses[i][0][j][0]))
        dic['a'].append(list(computed_responses[i][0][j][1]))
        dic['r'].append(list(computed_responses[i][0][j][2]))

with open("100_100_data.json", "w") as file:
    json.dump(dic, file)
'''

dic = {}
dic['c'], dic['a'], dic['r'] = [], [], []
for i in range(len(computed_responses)):
    for j in range(len(computed_responses[0])):
        dic['c'].append(list(computed_responses[i][j][0]))
        dic['a'].append(list(computed_responses[i][j][1]))
        dic['r'].append(list(computed_responses[i][j][2]))

with open("/content/1_10000_data.json", "w") as file:
    json.dump(dic, file)

In [69]:
sample = {}
for i in range(len(s)):
    sample[str(i)] = s[i].tolist()

with open("/content/1_10000_sample.json", "w") as file:
    json.dump(sample, file)

## Normalization

In [2]:
def min_max(data):
    data = np.array(data)
    col_min = np.min(data, axis=0)
    col_max = np.max(data, axis=0)
    return np.array(col_min), np.array(col_max)

def normalize_data(data, dmin, dmax):
    data = np.array(data)
    dmin = np.array(dmin)
    dmax = np.array(dmax)
    return np.nan_to_num((data - dmin)/(dmax-dmin), nan=0)

def denormalize_data(data, dmin, dmax):
    data = np.array(data)
    dmin = np.array(dmin)
    dmax = np.array(dmax)
    denorm = data * (dmax-dmin) + dmin
    return denorm

In [3]:
with open("100_100_data.json", "r") as file:
    data = json.load(file)

min_c, max_c = min_max(data['c'])
min_a, max_a = min_max(data['a'])
min_r, max_r = min_max(data['r'])

n_c = normalize_data(data['c'], min_c, max_c)
n_a = normalize_data(data['a'], min_a, max_a)
n_r = normalize_data(data['c'], min_r, max_r)

/var/folders/2m/9sw7l7t1625_t5p1j53brwx80000gn/T/ipykernel_87741/707544930.py:11: RuntimeWarning: invalid value encountered in true_divide
  return np.nan_to_num((data - dmin)/(dmax-dmin), nan=0)


In [6]:
with open("100_100_sample.json", "r") as file:
    samples = json.load(file)

sample = []
for i in samples.keys():
    for j in range(100):
        sample.append(samples[i])

bound = np.asarray([50.0, 500.0, 0.01, 50.0, 50.0, 5.0, 10.0, 0.5, 1.0, 0.2, 1.0, 1.0, 2.0, 50.0, 100.0])
dmin = bound * 0.1
dmax = bound * 2.0

target = normalize_data(sample, dmin, dmax)

In [7]:
len(target)

10000

## Pre-processing data to fit in the model

In [8]:
All_data = []
for i in range(len(n_c)):
    All_data.append(np.array([n_c[i], n_a[i], n_r[i]]))

All_parameter = []
for i in range(len(n_c)):
    All_parameter.append(target[i])

In [9]:
x_train, df1, y_train, df2 = train_test_split(
    All_data, All_parameter,
    test_size = 0.1,
    random_state = RANDOM_SEED
)

x_val, x_test, y_val, y_test = train_test_split(
    df1, df2,
    test_size = 0.5,
    random_state = RANDOM_SEED
)

In [10]:
train_dataset = TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())
val_dataset = TensorDataset(torch.tensor(x_val).float(), torch.tensor(y_val).float())
test_dataset = TensorDataset(torch.tensor(x_test).float(), torch.tensor(y_test).float())
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)

/var/folders/2m/9sw7l7t1625_t5p1j53brwx80000gn/T/ipykernel_87741/1790714107.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  train_dataset = TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())


## CNN

In [11]:
class MyCNN(nn.Module):
    def __init__(self, seq_len=201, n_features=3):
        super(MyCNN, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features

        self.layer1 = nn.Sequential(
            nn.Conv1d(3, 25, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(25, 25, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3))

        self.layer2 = nn.Sequential(
            nn.Conv1d(25, 50, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(50, 50, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3))
        
        self.layer3 = nn.Sequential(
            nn.Conv1d(50, 100, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(100, 100, kernel_size=3, stride=1, padding=1),
            nn.ReLU())

        self.layer4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=22),
            nn.Flatten())

        self.layer5 = nn.Sequential(
            nn.Linear(100, 100),
            nn.BatchNorm1d(num_features=100, momentum=0.99))

        self.layer6 = nn.Sequential(
            nn.Linear(100, 100),
            nn.BatchNorm1d(num_features=100, momentum=0.99))

        self.fc = nn.Linear(100, 15)

    def forward(self, x):
        y = self.layer1(x)
        y = self.layer2(y)
        y = self.layer3(y)
        y = self.layer4(y)
        y = self.layer5(y)
        y = self.layer6(y)
        y = self.fc(y)
        return y

In [12]:
class CNN_model(nn.Module):
    def __init__(self, seq_len=201, n_features=3):
        super(CNN_model, self).__init__()
        self.encoder = MyCNN(seq_len, n_features).to(device)


    def forward(self, x):
        y = self.encoder(x)
        #print(y)
        return y

In [53]:
model = CNN_model().to(device)

## Model training

In [79]:
def train_model(model, x_train, x_val, y_train, y_val, n_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss(reduction='sum').to(device)
    history = dict(train=[], val=[])

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000.0
    
    for epoch in range(1, n_epochs + 1):
        start_time = time.time()
        model = model.train()
        train_losses = []
        for batch_idx, (data, target) in enumerate(train_loader):
        #for (data, target) in train_dataset:
        #for i in range(len(x_train)):
            optimizer.zero_grad()
            #output = model(torch.tensor(x_train[i]).float()).squeeze().to(device)
            output = model(data)
            #print(output, y_train[i][0])
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
            #for (data, target) in val_dataset:
            #for i in range(len(x_val)):
                #output = model(torch.tensor(x_val[i]).float()).squeeze().to(device)
                output = model(data)
                loss = criterion(output, target)
                val_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if (early_stop_counter >= 10) and (val_loss-best_loss >= 0.05):
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
        print('time: ', time.time()-start_time)

    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [80]:
model, history = train_model(
    model, 
    x_train, x_val,
    y_train, y_val,
    n_epochs = 500
)

Epoch 1: train loss 1.2896561116245058 val loss 1.2786007167100906
time:  27.94154715538025
Epoch 2: train loss 1.2639528710875245 val loss 1.2687218658924102
time:  28.40338659286499
Epoch 3: train loss 1.2586407295299902 val loss 1.2650325590372085
time:  28.13886785507202
Epoch 4: train loss 1.2564020778470568 val loss 1.263955364227295
time:  27.768731355667114
Epoch 5: train loss 1.2557914857168992 val loss 1.263009960591793
time:  27.395764350891113


In [13]:
#with open('CNN_100_100_history.json', 'w') as f:
    #json.dump(history, f)

#with open("tcn_100_100_history.json", "r") as file:
    #history = json.load(file)
#torch.save(model, 'cnn_nor.pth')
model = torch.load('cnn.pth') 

In [34]:
original, predict = [], []
for batch_idx, (data, target) in enumerate(test_loader):
    output = model(data.to(device))
    for i in output:
        predict.append(i.detach().numpy())
    for j in target:
        original.append(j.detach().numpy())

## MAE

In [35]:
mae = np.mean(np.abs(np.array(original) - np.array(predict)))
mae

0.25874043